In [2]:
import numpy as np
import pandas as pd
from keras.utils import image_dataset_from_directory

In [3]:
df="/kaggle/input/persian-handwritten-digits/Train"

In [4]:
df

'/kaggle/input/persian-handwritten-digits/Train'

In [5]:
train_data=image_dataset_from_directory(
df,
image_size=(256,256),
batch_size=32,
label_mode=None,
)

Found 100000 files.


# # GAN

In [11]:
from keras.layers import Dense,Flatten,BatchNormalization,Dropout,ReLU,Reshape,Conv2DTranspose,Conv2D,MaxPooling2D,Input
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [46]:
def build_gen():
    inputs=Input(shape=(100,))
    C=Dense(16*16*256)(inputs)
    C=Reshape((16,16,256))(C)
    C=Conv2DTranspose(256,(3,3),strides=(2,2),padding="same")(C)
    C=ReLU()(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)

    C=Conv2DTranspose(128,(3,3),strides=(2,2),padding="same",kernel_initializer="HeNormal",kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)

    C=Conv2DTranspose(64,(3,3),strides=(2,2),activation="relu",padding='same')(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)

    C=Conv2DTranspose(32,(3,3),strides=(2,2),padding="same",kernel_initializer="HeNormal",kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)
    C=Conv2D(3,(3,3),activation="relu",padding="same")(C)
    model=Model(inputs,C)
    return model
gen=build_gen()
gen.summary()

Model: "functional_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_21 (InputLayer)     │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 65536)          │     6,619,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_10 (Reshape)            │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_28             │ (None, 32, 32, 256)    │       590,080 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_27 (ReLU)                 │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_44 (Dropout)            │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_44          │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_29             │ (None, 64, 64, 128)    │       295,040 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_28 (ReLU)                 │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_45 (Dropout)            │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_45          │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_30             │ (None, 128, 128, 64)   │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_46 (Dropout)            │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_46          │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_31             │ (None, 256, 256, 32)   │        18,464 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_29 (ReLU)                 │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_47 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_47          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 256, 256, 3)    │           867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,599,299 (28.99 MB)

 Trainable params: 7,598,339 (28.99 MB)

 Non-trainable params: 960 (3.75 KB)

In [47]:
def build_dis():
    inputs=Input(shape=(256,256,3))
    C=Conv2D(32,(3,3),activation="relu",padding='same')(inputs)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)
    C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)

    C=Conv2D(64,(3,3),padding="same",kernel_initializer="HeNormal",kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)
    C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)

    C=Conv2D(128,(3,3),activation="relu",padding='same')(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)
    C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)

    C=Conv2D(256,(3,3),padding="same",kernel_initializer="HeNormal",kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)
    C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)

    C=Flatten()(C)
    D=Dense(32,activation="relu")(C)
    D=Dense(1,activation="sigmoid")(D)
    model=Model(inputs,D)
    return model
dis=build_dis()
dis.summary()

Model: "functional_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_22 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_48 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_48          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_30 (ReLU)                 │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_49 (Dropout)            │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_49          │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_50 (Dropout)            │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_50          │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_31 (ReLU)                 │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_51 (Dropout)            │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_51          │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 65536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 32)             │     2,097,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,487,553 (9.49 MB)

 Trainable params: 2,486,593 (9.49 MB)

 Non-trainable params: 960 (3.75 KB)

In [48]:
def build_gan(generator,discriminator,input_shape):
    dis.trainable=False
    inputs=Input(shape=(input_shape,))
    fake_img=generator(inputs)
    outputs=discriminator(fake_img)
    model=Model(inputs,outputs)
    return model
gan=build_gan(generator=gen,discriminator=dis,input_shape=100)
gan.summary()

Model: "functional_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_23 (InputLayer)     │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_21 (Functional)      │ (None, 256, 256, 3)    │     7,599,299 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_23 (Functional)      │ (None, 1)              │     2,487,553 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,086,852 (38.48 MB)

 Trainable params: 7,598,339 (28.99 MB)

 Non-trainable params: 2,488,513 (9.49 MB)

In [53]:
def train_gan(gan,generator,discriminator,epochs,batch_size,latent_dim,dataset):
    discriminator.trainable=True
    discriminator.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
    gan.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
    for epoch in range(epochs):
        for real_img in dataset:
            real_label=np.ones((batch_size,1))
            fake_label=np.ones((batch_size,1))
            noise=np.random.normal(0,1,(batch_size,latent_dim))
            fake_img=generator(noise)
            discriminator.trainable=True
            discriminator.train_on_batch(real_img,real_label)
            discriminator.train_on_batch(fake_img,fake_label)
            discriminator.trainable=False
            g_g=gan.train_on_batch(noise,real_label)
    return g_g


In [ ]:
train_gan=train_gan(gan,gen,dis,3,32,100,train_data)